### **Task 1**: Compare lower-bound with the actual cost

Details of the task:
- Set LP using [gurobi](https://www.gurobi.com/features/academic-named-user-license/): Need to set up my license

    Using [Google OR-Tools](https://developers.google.com/optimization/lp/lp_example) instead
- The lower bound is predicted by the LP objective function
- The actual solution from the given data OR with an ILP solver
- Reference [here](https://github.com/algo-cancer/PhISCS-BnB) for the data & optimal solution

**Questions**:
* Why does the input data have question marks? - make it automatically 0
* When can a mutation be "eliminated" (See the [Read Me Section](https://github.com/algo-cancer/PhISCS-BnB/tree/master?tab=readme-ov-file#output))?


### **Task 2**: Does the lower bound increase when adding constraints for non-conflict column pairs to the LP (empiricconflicty tested)

Details of the task:
- First test with the given data [here](https://github.com/algo-cancer/PhISCS-BnB)
- Then test with randomly generated data
- Compare lower bound with all pairs vs. only conflict pairs in the LP
- Is there 1/2 across the board that is lowering the bound of only conflict LPs?
- If the additional constraints don’t help - why?

**Questions**:
* Should I remove constraints (2), (3), (4), (5), (6)

    Effectively remove all variables $B_{p,q,x_1,x_2}$ if $p$ and $q$ don't have a conflict

### **Task 3**: Write up proof claiming that once a conflict is resolved, those 2 columns will be <, >, or ≠ in the final solution

Look at scratch.txt for additional ideas

## Functions

* `make_random_data(rows, cols, file, bias)` - create random SCS data with a bias
* `read_data(file)` - read the data from the file
* `get_conversion_cost(X, Y)` - calculate the number of mutations needed to convert X to Y
* `find_conflict_columns(X)` - matrix describing if a given pair of columns have a conflict
* `solve_LP(SCS, ColSelector = None, verbose = False)` - solve the LP with all pairs or conflict pairs
* `compare_SCS(SCS_array, SCS_names)` - compare SCS data among multiples DataFrames

In [52]:
# Generate random data
# Input: rows, cols, file, bias
# Output: random_data
def make_random_data(rows, cols, file=None, bias=0.7):
    random_data = [([f'cell{i}'] if file else []) + [int(random.uniform(0, 1) < bias) for j in range(cols)] for i in range(rows)]
    random_data = pd.DataFrame(random_data, columns=(['cellIDxmutID'] if file else []) + [f'mut{i}' for i in range(cols)])
    if file:
        random_data.to_csv(file + ".SC", index=False, sep='\t')
    return random_data

In [53]:
# Read data function
# Input: file - name of the file without extension
# Return: In_SCS, CF_SCS, MutsAtEdges
# Note: SCS converts all ? to 0
# Note: MutsAtEdges is a list with a tuple - (parent, curr_node, muts: set)
find_nodes_re = r"\[(?P<parent>[0-9]+)\]->\[(?P<node>[0-9]+)\]:"
def read_data(file):
    raw = pd.read_csv(file + ".SC", sep="\t", dtype=str)
    In_SCS = (raw.iloc[:, 1:] == "1").astype(np.bool)
    try:
        CF_SCS = pd.read_csv(file + ".CFMatrix", sep="\t").iloc[:, 1:]
    except:
        CF_SCS = None
    try:
        MutsAtEdges = []
        with open(file + ".mutsAtEdges", "r") as f:
            for line in f:
                l = line.strip().split(' ')
                parent, curr_node = tuple(map(int, re.match(find_nodes_re, l[0]).groups()))
                MutsAtEdges.append((parent, curr_node, set(l[1:])))
    except:
        MutsAtEdges = None
    return In_SCS, CF_SCS, MutsAtEdges

In [54]:
# Conversion cost
# Input: X - from matrix, Y - to matrix
# Return: Cost of converting X into Y
def get_conversion_cost(X, Y):
    cost = 0
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            if X.iloc[i, j] > Y.iloc[i, j]:
                exit(1) # This is a false positive mutation
            if X.iloc[i, j] < Y.iloc[i, j]:
                cost += 1
    return cost

In [55]:
# Conflict column pairs
# Input: X - matrix of SCS data
# Return: nxn columns pairs, True - is conflict

# Check if a column pair has conflicts - Utility function
def is_conflict(df, p, q):
    is10 = False
    is01 = False
    is11 = False
    for k in range(df.shape[0]):
        if df.iloc[k, p] == 1 and df.iloc[k, q] == 0:
            is10 = True
        if df.iloc[k, p] == 0 and df.iloc[k, q] == 1:
            is01 = True
        if df.iloc[k, p] == 1 and df.iloc[k, q] == 1:
            is11 = True
    return is10 and is01 and is11

# Get matrix of is_conflict
def find_conflict_columns(X):
    conflicts = []
    for p in range(n):
        temp = []
        for q in range(n):
            temp.append(is_conflict(X, p, q))
        conflicts.append(temp)
    conflicts = pd.DataFrame(conflicts)
    return conflicts
# TODO: Make this more efficient - how?

In [56]:
# Solve the LP & find the lower bound
# Input: SCS, ColSelector - which col pairs to add constraints for, verbose
# Return: LP_objective, LP_solution (float)
def solve_LP(SCS, ColSelector = None, verbose = False):
    
    solver = pywraplp.Solver.CreateSolver("GLOP")
    m = SCS.shape[0] # rows
    n = SCS.shape[1] # cols

    # Create variables
    vars = {}
    for p in range(n):
        for q in range(p+1, n):
            if ColSelector is None or ColSelector.iloc[p, q]: # Check cols
                vars[f"B_{p}_{q}_1_0"] = solver.NumVar(0, 1, f"B_{p}_{q}_1_0") # (6)
                vars[f"B_{p}_{q}_0_1"] = solver.NumVar(0, 1, f"B_{p}_{q}_0_1") # (6)
                vars[f"B_{p}_{q}_1_1"] = solver.NumVar(0, 1, f"B_{p}_{q}_1_1") # (6)
    for i in range(m):
        for j in range(n):
            vars[f"x_{i}_{j}"] = solver.NumVar(float(SCS.iloc[i, j]), 1, f"x_{i}_{j}") # (7)
    if verbose:
        print(solver.NumVariables(), "variables created")

    # Create constraints
    for p in range(n):
        for q in range(p+1, n):
            if ColSelector is None or ColSelector.iloc[p, q]: # Check cols
                solver.Add(vars[f"B_{p}_{q}_1_0"] + vars[f"B_{p}_{q}_0_1"] + vars[f"B_{p}_{q}_1_1"] <= 2) # (5)
                for i in range(m):
                    solver.Add(vars[f"x_{i}_{p}"] - vars[f"x_{i}_{q}"] <= vars[f"B_{p}_{q}_1_0"]) # (2)
                    solver.Add(- vars[f"x_{i}_{p}"] + vars[f"x_{i}_{q}"] <= vars[f"B_{p}_{q}_0_1"]) # (3)
                    solver.Add(vars[f"x_{i}_{p}"] + vars[f"x_{i}_{q}"] <= 1 + vars[f"B_{p}_{q}_1_1"]) # (4)
    if verbose:
        print(solver.NumConstraints(), "constraints created")

    # Define objective function
    objective = solver.Objective()
    for i in range(m):
        for j in range(n):
            if SCS.iloc[i, j] == 0: # only if they used to be 0
                objective.SetCoefficient(vars[f"x_{i}_{j}"], 1) # (1)
    objective.SetMinimization()

    # Solve & print objective
    status = solver.Solve()
    if status != pywraplp.Solver.OPTIMAL:
        print("The problem does not have an optimal solution.")
        exit(1)
    objective_value = objective.Value()
    if verbose:
        print(f"Solving with {solver.SolverVersion()}\n")
        print(f"Solution:\nLower bound (LP objective) = {objective_value:0.5f}")

    # Create & print the solution DF
    solution = []
    for i in range(m):
        solution.append([vars[f"x_{i}_{j}"].solution_value() for j in range(n)])
    solution = pd.DataFrame(solution)
    if verbose:
        display(solution)

    # Return
    return objective_value, solution

In [57]:
# Compare SCS data
# Input: SCS_array - list of SCS matrices, SCS_names - list of names
# Return: Matrix of each difference - (row, col, mat1_val, mat2_val, ...)
def compare_SCS(SCS_array, SCS_names):
    m = SCS_array[0].shape[0]
    n = SCS_array[0].shape[1]
    diffs = []
    for i in range(m):
        for j in range(n):
            to_add = False
            temp = [i, j]
            for SCS_DF in SCS_array:
                if SCS_DF.iloc[i, j] != SCS_array[0].iloc[i, j]:
                    to_add = True
                temp.append(SCS_DF.iloc[i, j])
            if to_add:
                diffs.append(temp)
    diffs = pd.DataFrame(diffs, columns=["row", "col"] + SCS_names)
    return diffs

## Driver

In [58]:
# Imports
import pandas as pd
from ortools.linear_solver import pywraplp
from scphylo import datasets
import numpy as np
import datetime
import os
import re
import random
import time

from vc import vertex_cover_pp

In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
# Arguments
DISPLAY_TABLES = False # Display the output tables
READ_DATA = True # Read data from files
REAL_DATA = datasets.melanoma20().X # Dataset to use (None if no dataset)
FILE = "./example/data2" # File name without extension
# If Read_Data is False, a None file will write to no file
# If Read_Data is True (and ), FILE will be the file to read from (no extension)

# For creating random data
if not READ_DATA and REAL_DATA is None:
    m = 200 # rows
    n = 50 # cols
    # Note: m and n get defined later in other cases

exp_timers = {}  # track the experiments' durations

/home/juan/.pyenv/versions/phiscs/lib/python3.10/site-packages/anndata/__init__.py:52: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [61]:
# Option 1: Create data (Read_Data = False and )
if not READ_DATA and REAL_DATA is None:
    In_SCS = make_random_data(m, n, FILE)
    print("Dimensions of the created data:", In_SCS.shape)
    if DISPLAY_TABLES:
        display(In_SCS)

In [62]:
# Option 2: Get real data (Real_data exists)
if not READ_DATA and REAL_DATA is not None:

    # Other datasets: https://scphylo-tools.readthedocs.io/en/latest/api_reference.html#datasets-datasets
    In_SCS = pd.DataFrame(REAL_DATA).astype(int)
    In_SCS.columns = [f"mut{i}" for i in range(In_SCS.shape[1])]
    m = In_SCS.shape[0] # rows
    n = In_SCS.shape[1] # cols

    # NOTE: LOTS OF COLS, LITTLE DATA

    print(f"Data shape: {In_SCS.shape}")
    if DISPLAY_TABLES:
        display(In_SCS)

In [63]:
# Option 3: Read data (Read_Data = True)
if READ_DATA:

    In_SCS, CF_SCS, MutsAtEdges = read_data(FILE)
    m = In_SCS.shape[0] # rows
    n = In_SCS.shape[1] # cols
    print(f"Data shape: {In_SCS.shape}")
    if DISPLAY_TABLES:
        print("\nInput SCS data:")
        display(In_SCS)
        print("Conflict-free SCS (answer) data:")
        display(CF_SCS)

Data shape: (50, 50)


In [64]:
# Compare In_SCS and CF_SCS (if available)
if READ_DATA:
    real_cost = get_conversion_cost(In_SCS, CF_SCS)
    print(f"True cost of converting Input SCS to Conflict-Free SCS: {real_cost}\n")
    print("Mutations (only false negative) between Input SCS and Conflict-Free SCS:")
    if DISPLAY_TABLES:
        display(compare_SCS([In_SCS, CF_SCS], ["In_SCS", "CF_SCS"]))

True cost of converting Input SCS to Conflict-Free SCS: 40

Mutations (only false negative) between Input SCS and Conflict-Free SCS:


In [65]:
# Task 1.1: Find the LP based lower bound (all columns)
exp_timers['time_solve_lp_all_cols'] = time.time()
LP_bound_all_columns, LP_solution_all_columns = solve_LP(In_SCS)
exp_timers['time_solve_lp_all_cols'] = time.time() - exp_timers['time_solve_lp_all_cols']
print("Lower bound (LP objective) with all columns:", LP_bound_all_columns)

# Tasks 1.2 Find the Vertex Cover based lower bound
exp_timers['time_solve_vc'] = time.time()
vc_lb, vc_flipped_bits = vertex_cover_pp(In_SCS.to_numpy())
exp_timers['time_solve_vc'] = time.time() - exp_timers['time_solve_vc']
print(f"Lower bound (VC size / 2): {vc_lb}")

if DISPLAY_TABLES:
    display(LP_solution_all_columns)

Lower bound (LP objective) with all columns: 34.0
Lower bound (VC size / 2): 24


In [66]:
# Task 1: Deterministic rounding & compare
RoundedLP_solution_all_columns = (LP_solution_all_columns.iloc[:, :] >= 0.5).astype(int)
RoundedLP_cost_all_columns = get_conversion_cost(In_SCS, RoundedLP_solution_all_columns)
print(f"Cost of converting rounded solution for LP with all columns to Input SCS: {RoundedLP_cost_all_columns}")
if DISPLAY_TABLES:
    display(compare_SCS([In_SCS, RoundedLP_solution_all_columns], ["In_SCS", "Rounded All Columns"]))

Cost of converting rounded solution for LP with all columns to Input SCS: 50


In [67]:
# Task 2: Get the conflict columns - takes a long time
exp_timers['time_solve_lp_conflict_cols_only'] = time.time()
conflict_columns = find_conflict_columns(In_SCS)
if DISPLAY_TABLES:
    print("Conflict columns:")
    display(pd.DataFrame(conflict_columns))

In [68]:
# Task 2: Find the LP based lower bound (conflict columns)
LP_bound_conflict_columns, LP_solution_conflict_columns = solve_LP(In_SCS, conflict_columns)
print("Lower bound (LP objective) with conflict columns:", LP_bound_conflict_columns)
if DISPLAY_TABLES:
    display(LP_solution_conflict_columns)
exp_timers['time_solve_lp_conflict_cols_only'] = time.time() - exp_timers['time_solve_lp_conflict_cols_only'] 

Lower bound (LP objective) with conflict columns: 34.0


In [69]:
# Task 2: Deterministic rounding & compare
RoundedLP_solution_conflict_columns = (LP_solution_conflict_columns.iloc[:, :] >= 0.5).astype(int)
RoundedLP_cost_conflict_columns = get_conversion_cost(In_SCS, RoundedLP_solution_conflict_columns)
print(f"Cost of converting rounded solution for LP with conflict columns to Input SCS: {RoundedLP_cost_conflict_columns}")
if DISPLAY_TABLES:
    display(compare_SCS([In_SCS, RoundedLP_solution_conflict_columns], ["In_SCS", "Rounded Conflict Columns"]))

Cost of converting rounded solution for LP with conflict columns to Input SCS: 50


In [70]:
# Compare costs
if READ_DATA:
    print(f"True cost of converting Input SCS to Conflict-Free SCS: {real_cost}")
print("Lower bound (LP objective) with all columns:", LP_bound_all_columns)
print("Lower bound (LP objective) with conflict columns:", LP_bound_conflict_columns)
print(f"Cost of converting rounded solution for LP with all columns to Input SCS: {RoundedLP_cost_all_columns}")
print(f"Cost of converting rounded solution for LP with conflict columns to Input SCS: {RoundedLP_cost_conflict_columns}")

True cost of converting Input SCS to Conflict-Free SCS: 40
Lower bound (LP objective) with all columns: 34.0
Lower bound (LP objective) with conflict columns: 34.0
Cost of converting rounded solution for LP with all columns to Input SCS: 50
Cost of converting rounded solution for LP with conflict columns to Input SCS: 50


In [71]:
# Compare LP (all columns) and LP (conflict columns)
print("LP (all columns) vs LP (conflict columns):")
display(compare_SCS([LP_solution_all_columns, LP_solution_conflict_columns], ["LP_all_columns", "LP_conflict_columns"]))

LP (all columns) vs LP (conflict columns):


,row,col,LP_all_columns,LP_conflict_columns


In [72]:
# Compare Rounded LP (all columns) and Rounded LP (conflict columns)
print("Rounded LP (all columns) vs Rounded LP (conflict columns):")
display(compare_SCS([RoundedLP_solution_all_columns, RoundedLP_solution_conflict_columns],
    ["RoundedLP_all_columns", "RoundedLP_conflict_columns"]))

Rounded LP (all columns) vs Rounded LP (conflict columns):


,row,col,RoundedLP_all_columns,RoundedLP_conflict_columns


In [73]:
# Write results to disk
results = pd.DataFrame(
    {
        'lp_obj_all_columns': [LP_bound_all_columns],
        'lp_obj_conflict_columns_only': [LP_bound_conflict_columns],
        'lp_sol_all_columns': [LP_solution_all_columns],
        'lp_sol_conflict_columns_only': [LP_solution_conflict_columns],
        'rounded_lp_obj_all_columns': [RoundedLP_cost_all_columns],
        'rounded_lp_obj_conflict_columns_only': [RoundedLP_cost_conflict_columns],
        'rounded_lp_sol_all_columns': [RoundedLP_solution_all_columns],
        'rounded_lp_sol_conflict_columns_only': [RoundedLP_solution_conflict_columns],
        'vc_lb': [vc_lb],
        'vc_flipped_bits': [vc_flipped_bits],
    } 
    | exp_timers
)

In [74]:
EXPS_DIR = 'results'
write_time_str = str(datetime.datetime.now().replace(microsecond=0))
CSV_PATH = os.path.join(EXPS_DIR, write_time_str + '.csv')

if not os.path.exists(EXPS_DIR):
    os.mkdir(EXPS_DIR)
results.to_csv(CSV_PATH)

## TODO

 - confernec

mailto:farid.rashidimehrabadi@nih.gov - ask for data

Make sure to CC Salem (and Cenk)

Tree visualization

incoporate mutsAtEdges

Add the list of relevent data points to the driver section